In [22]:
import os
import pandas as pd
import numpy as np
from itertools import product, combinations
import transformers
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from transformers import LongformerTokenizerFast, \
LongformerModel, LongformerTokenizer, LongformerConfig, Trainer, TrainingArguments, EvalPrediction, AutoTokenizer, AutoModel
from transformers.models.longformer.modeling_longformer import LongformerPreTrainedModel, LongformerClassificationHead
from sentence_transformers import SentenceTransformer
from tensorflow.keras import layers
from tensorflow.keras import losses

In [23]:
directory = "patterns/ProjectManagmentPatternLanguage"
starting_contexts = []
resulting_contexts = []
pattern_names = []
pattern_languages = []
for filename in os.listdir(directory):
    print(filename)
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("ProjectManagmentPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))
directory = "patterns/PiecemealGrowthPatternLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("PiecemealGrowthPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))

directory = "patterns/OrganizationalStyleLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("OrganizationalStyleLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))
        
        
directory = "patterns/PeopleAndCodePatternLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("PeopleAndCodePatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))

BuildPrototypes.txt
CommunityOfTrust.txt
CompletionHeadroom.txt
DayCare.txt
DeveloperControlsProcess.txt
DevelopmentEpisode.txt
DontInteruptAnInterrupt.txt
GetOnWithIt.txt
ImpliedRequirements.txt
IncrementalIntegration.txt
InformalLaborPlan.txt
InterruptsUnjamBlocking.txt
MercenaryAnalyst.txt
NamedStableBases.txt
PrivateWorld.txt
ProgrammingEpisode.txt
RecommitmentMeeting.txt
SacrificeOnePerson.txt
SizeTheSchedule.txt
SomeoneAlwaysMakesProgress.txt
TakeNoSmallSlips.txt
TeamPerTask.txt
WorkFlowsInward.txt
WorkQueue.txt
WorkSplit.txt


In [24]:
patterns=[]
for i,pattern_name in enumerate(pattern_names):
    patterns.append(pattern_name)
    patterns.append(starting_contexts[i])
    patterns.append(resulting_contexts[i])

In [25]:
d = {"pattern_names": pattern_names, 'pattern_language': pattern_languages,'starting_contexts': starting_contexts, "resulting_contexts": resulting_contexts}
df = pd.DataFrame(data=d)

In [26]:
df

,pattern_names,pattern_language,starting_contexts,resulting_contexts
0,Build Prototypes,ProjectManagmentPatternLanguage,A project must test requirements and design de...,Build an isolated prototype solution whose pur...
1,Community Of Trust,ProjectManagmentPatternLanguage,It is essential that the people in a team trus...,"Do things that explicitly demonstrate trust, s..."
2,Completion Headroom,ProjectManagmentPatternLanguage,Every project must commit to delivery on a few...,Project work group completion dates from remai...
3,Day Care,ProjectManagmentPatternLanguage,Your experts are spending all their time mento...,"Put one expert in charge of all the novices, l..."
4,Developer Controls Process,ProjectManagmentPatternLanguage,"A development culture, like any culture, can b...",Make the Developer the focal point of process ...
...,...,...,...,...
87,Smoke-Filled Room,PeopleAndCodePatternLanguage,An organization must make a timely decision ab...,Make the decision among power brokers as in th...
88,Standards Linking Locations,PeopleAndCodePatternLanguage,It is difficult for geographically distanced t...,Use standards to express architectural concern...
89,Stand-Up Meeting,PeopleAndCodePatternLanguage,"At times of fast change or high stress, it is ...",Hold short daily meetings with the entire team...
90,Subclass Per Team,PeopleAndCodePatternLanguage,Subsystem teams have differing interests and d...,"Where two subsystems collide in one class, fac..."


In [27]:
model = SentenceTransformer('all-mpnet-base-v2')

df['starting_embedding'] = df['starting_contexts'].apply(model.encode)
df['resulting_embedding'] = df['resulting_contexts'].apply(model.encode)

In [28]:
pattern_pairs = list(combinations(df['pattern_names'].tolist(),2))
df_2 = pd.DataFrame(list(combinations(df['pattern_names'].tolist(),2)), columns=['pattern_name1','pattern_name2'])
df_2["resulting_embedding1"] = df_2.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].resulting_embedding.item(), axis=1)
df_2["resulting_embedding2"] = df_2.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].resulting_embedding.item(), axis=1)
df_2["starting_embedding1"] = df_2.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].starting_embedding.item(), axis=1)
df_2["starting_embedding2"] = df_2.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].starting_embedding.item(), axis=1)
df_2

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0..."
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08..."
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0...."
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080..."
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0..."
...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4182,Standards Linking Locations,Variation Behind Interface,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.011085069, 0.045864385, -0.03685276, -0.008..."
4183,Stand-Up Meeting,Subclass Per Team,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4184,Stand-Up Meeting,Variation Behind Interface,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.011085069, 0.045864385, -0.03685276, -0.008..."


In [29]:
starting_embedding1_weighted = []
starting_embedding2_weighted = []
resulting_embedding1_weighted = []
resulting_embedding2_weighted = []

starting_embedding1_binary = []
starting_embedding2_binary = []
resulting_embedding1_binary = []
resulting_embedding2_binary = []
    
for i in range (len(df_2)):
    vector1 = df_2.iloc[i]['starting_embedding1'].copy()
    vector2 = df_2.iloc[i]['starting_embedding2'].copy()
    vector3 = df_2.iloc[i]['resulting_embedding1'].copy()
    vector4 = df_2.iloc[i]['resulting_embedding2'].copy()
    
    vector5 = df_2.iloc[i]['starting_embedding1'].copy()
    vector6 = df_2.iloc[i]['starting_embedding2'].copy()
    vector7 = df_2.iloc[i]['resulting_embedding1'].copy()
    vector8 = df_2.iloc[i]['resulting_embedding2'].copy()
    for j in range(768):
        position = j/768
        
        if vector5[j] > 0: 
            vector5[j] = 1 
        else: vector5[j] = 0
            
        if vector6[j] > 0: 
            vector6[j] = 1 
        else: vector6[j] = 0
            
        if vector7[j] > 0: 
            vector7[j] = 1 
        else: vector7[j] = 0
            
        if vector8[j] > 0: 
            vector8[j] = 1 
        else: vector8[j] = 0
                
        if position <=0.25:
            vector1[j] = vector1[j]*3
            vector2[j] = vector2[j]*3
            vector3[j] = vector3[j]*3
            vector4[j] = vector4[j]*3
        if position >= 0.75:
            vector1[j] = vector1[j]*2
            vector2[j] = vector2[j]*2
            vector3[j] = vector3[j]*2
            vector4[j] = vector4[j]*2
    starting_embedding1_weighted.append(vector1)
    starting_embedding2_weighted.append(vector2)
    resulting_embedding1_weighted.append(vector3)
    resulting_embedding2_weighted.append(vector4)
    
    starting_embedding1_binary.append(vector5)
    starting_embedding2_binary.append(vector6)
    resulting_embedding1_binary.append(vector7)
    resulting_embedding2_binary.append(vector8)
    

df_2['starting_embedding1_weighted'] = starting_embedding1_weighted
df_2['starting_embedding2_weighted'] = starting_embedding2_weighted
df_2['resulting_embedding1_weighted'] = resulting_embedding1_weighted
df_2['resulting_embedding2_weighted'] = resulting_embedding2_weighted

df_2['starting_embedding1_binary'] = starting_embedding1_binary
df_2['starting_embedding2_binary'] = starting_embedding2_binary
df_2['resulting_embedding1_binary'] = resulting_embedding1_binary
df_2['resulting_embedding2_binary'] = resulting_embedding2_binary

df_2

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,starting_embedding1_weighted,starting_embedding2_weighted,resulting_embedding1_weighted,resulting_embedding2_weighted,starting_embedding1_binary,starting_embedding2_binary,resulting_embedding1_binary,resulting_embedding2_binary
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1019777, 0.0065517956, -0.019318819, 0.1118...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.04167674, 0.2033511, -0.035876215, 0.042800...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[-0.19298586, 0.13900492, -0.047551688, -0.269...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[-0.08644326, 0.31615844, -0.017576242, -0.053...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.20991553, 0.16970119, -0.00050233543, -0.02...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.21322215, -0.010242404, -0.11686508, 0.0360...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ..."
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1487357, 0.17005202, -0.13856149, -0.024199...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.12500748, -0.1705819, -0.080992654, -0.1015...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.010742981, 0.003801091, -0.05702935, 0.0302...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.14548732, 0.038433768, -0.05634661, -0.0111...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840...","[0.14429173, -0.00025473803, -0

In [30]:
df_final = df_2.copy()
df_final

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,starting_embedding1_weighted,starting_embedding2_weighted,resulting_embedding1_weighted,resulting_embedding2_weighted,starting_embedding1_binary,starting_embedding2_binary,resulting_embedding1_binary,resulting_embedding2_binary
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1019777, 0.0065517956, -0.019318819, 0.1118...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.04167674, 0.2033511, -0.035876215, 0.042800...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[-0.19298586, 0.13900492, -0.047551688, -0.269...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[-0.08644326, 0.31615844, -0.017576242, -0.053...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.20991553, 0.16970119, -0.00050233543, -0.02...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.21322215, -0.010242404, -0.11686508, 0.0360...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, ..."
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1487357, 0.17005202, -0.13856149, -0.024199...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.12500748, -0.1705819, -0.080992654, -0.1015...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.010742981, 0.003801091, -0.05702935, 0.0302...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.14548732, 0.038433768, -0.05634661, -0.0111...","[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840...","[0.14429173, -0.00025473803, -0

In [31]:
#počítanie kosínusovej podobnosti

cos_sim1 = []
for pairs in df_final[['resulting_embedding1','starting_embedding2']].values:
    cos_sim1.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values1 = []
for values in cos_sim1:
    cos_values1.append(values[0][1])
    
cos_sim2 = []
for pairs in df_final[['starting_embedding1','starting_embedding2']].values:
    cos_sim2.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values2 = []
for values in cos_sim2:
    cos_values2.append(values[0][1])
    
cos_sim3 = []
for pairs in df_final[['resulting_embedding1','resulting_embedding2']].values:
    cos_sim3.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values3 = []
for values in cos_sim3:
    cos_values3.append(values[0][1])
    
cos_sim4 = []
for pairs in df_final[['starting_embedding1','resulting_embedding2']].values:
    cos_sim4.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values4 = []
for values in cos_sim4:
    cos_values4.append(values[0][1])
    
cos_sim5 = []
for pairs in df_final[['resulting_embedding1_weighted','starting_embedding2_weighted']].values:
    cos_sim5.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values5 = []
for values in cos_sim5:
    cos_values5.append(values[0][1])
    
cos_sim6 = []
for pairs in df_final[['starting_embedding1_weighted','starting_embedding2_weighted']].values:
    cos_sim6.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values6 = []
for values in cos_sim6:
    cos_values6.append(values[0][1])
    
cos_sim7 = []
for pairs in df_final[['resulting_embedding1_weighted','resulting_embedding2_weighted']].values:
    cos_sim7.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values7 = []
for values in cos_sim7:
    cos_values7.append(values[0][1])
    
cos_sim8 = []
for pairs in df_final[['starting_embedding1_weighted','resulting_embedding2_weighted']].values:
    cos_sim8.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values8 = []
for values in cos_sim8:
    cos_values8.append(values[0][1])
    
    
cos_sim9 = []
for pairs in df_final[['starting_embedding1','resulting_embedding1']].values:
    cos_sim9.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values9 = []
for values in cos_sim9:
    cos_values9.append(values[0][1])
    
cos_sim10 = []
for pairs in df_final[['starting_embedding2','resulting_embedding2']].values:
    cos_sim10.append((cosine_similarity([pairs[0],pairs[1]])+1)/2)
cos_values10 = []
for values in cos_sim10:
    cos_values10.append(values[0][1])


In [32]:
#počítanie euklidovskej vzdialenosti

euc_dist1 = []
for pairs in df_final[['resulting_embedding1','starting_embedding2']].values:
    euc_dist1.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values1 = []
for values in euc_dist1:
    euc_values1.append(values[0][1])
    
euc_dist2 = []
for pairs in df_final[['starting_embedding1','starting_embedding2']].values:
    euc_dist2.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values2 = []
for values in euc_dist2:
    euc_values2.append(values[0][1])
    
euc_dist3 = []
for pairs in df_final[['resulting_embedding1','resulting_embedding2']].values:
    euc_dist3.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values3 = []
for values in euc_dist3:
    euc_values3.append(values[0][1])
    
euc_dist4 = []
for pairs in df_final[['starting_embedding1','resulting_embedding2']].values:
    euc_dist4.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values4 = []
for values in euc_dist4:
    euc_values4.append(values[0][1])
    
euc_dist5 = []
for pairs in df_final[['resulting_embedding1_weighted','starting_embedding2_weighted']].values:
    euc_dist5.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values5 = []
for values in euc_dist5:
    euc_values5.append(values[0][1])
    
euc_dist6 = []
for pairs in df_final[['starting_embedding1_weighted','starting_embedding2_weighted']].values:
    euc_dist6.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values6 = []
for values in euc_dist6:
    euc_values6.append(values[0][1])
    
euc_dist7 = []
for pairs in df_final[['resulting_embedding1_weighted','resulting_embedding2_weighted']].values:
    euc_dist7.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values7 = []
for values in euc_dist7:
    euc_values7.append(values[0][1])
    
euc_dist8 = []
for pairs in df_final[['starting_embedding1_weighted','resulting_embedding2_weighted']].values:
    euc_dist8.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values8 = []
for values in euc_dist8:
    euc_values8.append(values[0][1])
    
    
euc_dist9 = []
for pairs in df_final[['starting_embedding1','resulting_embedding1']].values:
    euc_dist9.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values9 = []
for values in euc_dist9:
    euc_values9.append(values[0][1])
    
euc_dist10 = []
for pairs in df_final[['starting_embedding2','resulting_embedding2']].values:
    euc_dist10.append(1/(1+euclidean_distances([pairs[0],pairs[1]])))
euc_values10 = []
for values in euc_dist10:
    euc_values10.append(values[0][1])


In [33]:
def hamming_distance(vector1, vector2):
    ham = 0
    for j in range(768):
        if vector1[j]!=vector2[j]:
            ham = ham+1
    return ham

h_dist1 = []
for pairs in df_final[['resulting_embedding1_binary','starting_embedding2_binary']].values:
    h_dist1.append(hamming_distance(pairs[0],pairs[1]))
h_values1 = []
for values in h_dist1:
    h_values1.append(values)
    
h_dist2 = []
for pairs in df_final[['starting_embedding1_binary','starting_embedding2_binary']].values:
    h_dist2.append(hamming_distance(pairs[0],pairs[1]))
h_values2 = []
for values in h_dist2:
    h_values2.append(values)
    
h_dist3 = []
for pairs in df_final[['resulting_embedding1_binary','resulting_embedding2_binary']].values:
    h_dist3.append(hamming_distance(pairs[0],pairs[1]))
h_values3 = []
for values in h_dist3:
    h_values3.append(values)
    
h_dist4 = []
for pairs in df_final[['starting_embedding1_binary','resulting_embedding2_binary']].values:
    h_dist4.append(hamming_distance(pairs[0],pairs[1]))
h_values4 = []
for values in h_dist4:
    h_values4.append(values)

In [34]:
df_final['cos_similarity_result_start_1']=cos_values1
df_final['cos_similarity_start_start']=cos_values2
df_final['cos_similarity_result_result']=cos_values3
df_final['cos_similarity_result_start_2']=cos_values4
df_final['cos_similarity_same_1']=cos_values9
df_final['cos_similarity_same_2']=cos_values10

df_final['euc_distance_result_start_1']=euc_values1
df_final['euc_distance_start_start']=euc_values2
df_final['euc_distance_result_result']=euc_values3
df_final['euc_distance_result_start_2']=euc_values4
df_final['euc_distance_same_1']=euc_values9
df_final['euc_distance_same_2']=euc_values10


df_final['cos_similarity_result_start_1_weighted']=cos_values5
df_final['cos_similarity_start_start_weighted']=cos_values6
df_final['cos_similarity_result_result_weighted']=cos_values7
df_final['cos_similarity_result_start_2_weighted']=cos_values8

df_final['euc_distance_result_start_1_weighted']=euc_values5
df_final['euc_distance_start_start_weighted']=euc_values6
df_final['euc_distance_result_result_weighted']=euc_values7
df_final['euc_distance_result_start_2_weighted']=euc_values8

df_final['hem_distance_result_start_1']=h_values1
df_final['hem_distance_start_start']=h_values2
df_final['hem_distance_result_result']=h_values3
df_final['hem_distance_result_start_2']=h_values4


In [35]:
df_final

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,starting_embedding1_weighted,starting_embedding2_weighted,resulting_embedding1_weighted,resulting_embedding2_weighted,...,cos_similarity_result_result_weighted,cos_similarity_result_start_2_weighted,euc_distance_result_start_1_weighted,euc_distance_start_start_weighted,euc_distance_result_result_weighted,euc_distance_result_start_2_weighted,hem_distance_result_start_1,hem_distance_start_start,hem_distance_result_result,hem_distance_result_start_2
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1019777, 0.0065517956, -0.019318819, 0.1118...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.04167674, 0.2033511, -0.035876215, 0.042800...",...,0.633504,0.663182,0.295255,0.312311,0.295351,0.308063,319,275,335,297
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[-0.19298586, 0.13900492, -0.047551688, -0.269...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[-0.08644326, 0.31615844, -0.017576242, -0.053...",...,0.620173,0.637398,0.283406,0.297404,0.289833,0.298322,350,320,320,310
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0....","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.20991553, 0.16970119, -0.00050233543, -0.02...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.21322215, -0.010242404, -0.11686508, 0.0360...",...,0.684717,0.621988,0.289326,0.293272,0.306964,0.291549,356,332,320,318
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.1487357, 0.17005202, -0.13856149, -0.024199...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.12500748, -0.1705819, -0.080992654, -0.1015...",...,0.631332,0.682716,0.305274,0.345871,0.290161,0.309445,325,267,323,303
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.15096118, 0.05705569, 0.016062377, -0.08791...","[0.010742981, 0.003801091, -0.05702935, 0.0302...","[0.07832036, 0.13272882, 0.019368187, 0.028240...","[0.14548732, 0.038433768, -0.05634661, -0.0111...",...,0.685617,0.678533,0.269797,0.284647,0.313659,0.315293,377,339,303,297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840...","[0.14429173, -0.00025473803, -0.076118976, 0.0...","[0.097566, 0.081386015, 0.053376283, 0.0581521...","[0.01853433, 0.13493305, 0.010961279, -0.03613...","[0.1363283, 0.18643074, -0.04684811, 0.0672895...",...,0.705896,0.712318,0.302245,0.313882,0.324391,0.326197,318,300,271,275
4182,Standards Linking Locations,Variation Behind Interface,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.0

In [36]:
df_final.to_csv("pattern_pairs.csv", encoding='utf-8')

In [20]:
df[['pattern_names','pattern_language']].to_csv("pattern_languages.csv",encoding='utf-8')